<a href="https://colab.research.google.com/github/arafat-hasan/FacebookManager/blob/master/two-classes/waste_sorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!date

In [0]:
!pip install torchviz
!pip install livelossplot

#### modeling pipeline:

1. Download and extract the images
2. Organize the images into different folders
3. Train model
4. Make and evaluate test predictions
5. Next steps

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate
from pathlib import Path
from glob2 import glob
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import os
import zipfile as zf
import shutil
import re
import seaborn as sns

## 1. Extract data

In [0]:
!wget https://github.com/garythung/trashnet/raw/master/data/dataset-resized.zip

First, we need to extract the contents of "dataset-resized.zip".

In [0]:
!ls -la
!pwd

In [0]:
files = zf.ZipFile("dataset-resized.zip",'r')
files.extractall()
files.close()

Once unzipped, the dataset-resized folder has six subfolders:

&nbsp;&nbsp;&nbsp;&nbsp; /dataset-resized <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /cardboard <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /glass <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /metal <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /paper <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /plastic <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /trash <br>


In [0]:
os.listdir(os.path.join(os.getcwd(),"dataset-resized"))

Now, We will create two directory in side dataset-resized named `digestible` and `indigestible`. After that, we have to move all images from `trash` directory to `digestible` directory and all images from rest other five directories to `indigestible` directory. And then we have to delete the empty six directories.

&nbsp;&nbsp;&nbsp;&nbsp; /dataset-resized <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /digestible <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /indigestible <br>



In [0]:
!rm dataset-resized.zip
!ls -la /content
!mkdir /content/dataset-resized/digestible/ /content/dataset-resized/indigestible/
!ls -la /content/dataset-resized/

In [0]:
!mv /content/dataset-resized/cardboard/* /content/dataset-resized/indigestible/
!mv /content/dataset-resized/glass/* /content/dataset-resized/indigestible/
!mv /content/dataset-resized/metal/* /content/dataset-resized/indigestible/
!mv /content/dataset-resized/paper/* /content/dataset-resized/indigestible/
!mv /content/dataset-resized/plastic/* /content/dataset-resized/indigestible/
!mv /content/dataset-resized/trash/* /content/dataset-resized/digestible/

!rm -r /content/dataset-resized/cardboard/
!rm -r /content/dataset-resized/glass/
!rm -r /content/dataset-resized/metal/
!rm -r /content/dataset-resized/paper/
!rm -r /content/dataset-resized/plastic/
!rm -r /content/dataset-resized/trash/

In [0]:
!ls -la /content/dataset-resized/

In [0]:
!ls -v /content/dataset-resized/digestible/ | cat -n | while read n f; do mv -n "/content/dataset-resized/digestible/$f" "/content/dataset-resized/digestible/digestible$n.jpg"; done 

In [0]:
!ls -v /content/dataset-resized/indigestible/ | cat -n | while read n f; do mv -n "/content/dataset-resized/indigestible/$f" "/content/dataset-resized/indigestible/indigestible$n.jpg"; done 

## 2. Organize images into different folders



In [0]:
## helper functions ##

## splits indices for a folder into train, validation, and test indices with random sampling
    ## input: folder path
    ## output: train, valid, and test indices    
def split_indices(folder,seed1,seed2):    
    n = len(os.listdir(folder))
    full_set = list(range(1,n+1))

    ## train indices
    random.seed(seed1)
    train = random.sample(list(range(1,n+1)),int(.8*n))

    ## temp
    remain = list(set(full_set)-set(train))

    ## separate remaining into validation and test
    random.seed(seed2)
    valid = random.sample(remain,int(.5*len(remain)))
    test = list(set(remain)-set(valid))
    
    return(train,valid,test)

## gets file names for a particular type of trash, given indices
    ## input: waste category and indices
    ## output: file names 
def get_names(waste_type,indices):
    file_names = [waste_type+str(i)+".jpg" for i in indices]
    return(file_names)    

## moves group of source files to another folder
    ## input: list of source files and destination folder
    ## no output
def move_files(source_files,destination_folder):
    for file in source_files:
        shutil.move(file,destination_folder)

Next, I'm going to create a bunch of destination folders according to the ImageNet directory convention. It'll look like this:

/data <br>
&nbsp;&nbsp;&nbsp;&nbsp; /train <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /digestible <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /indigestible <br>
&nbsp;&nbsp;&nbsp;&nbsp; /valid <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /digestible <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    /indigestible <br>
&nbsp;&nbsp;&nbsp;&nbsp;/test <br>

Each image file is just the material name and a number (i.e. digestible1.jpg)

Again, this is just housekeeping to organize my files.

In [0]:
## paths will be train/cardboard, train/glass, etc...
subsets = ['train','valid']
waste_types = ['digestible','indigestible']

## create destination folders for data subset and waste type
for subset in subsets:
    for waste_type in waste_types:
        folder = os.path.join('data',subset,waste_type)
        if not os.path.exists(folder):
            os.makedirs(folder)
            
if not os.path.exists(os.path.join('data','test')):
    os.makedirs(os.path.join('data','test'))
            
## move files to destination folders for each waste type
for waste_type in waste_types:
    source_folder = os.path.join('/content/dataset-resized',waste_type)
    train_ind, valid_ind, test_ind = split_indices(source_folder,1,1)
    
    ## move source files to train
    train_names = get_names(waste_type,train_ind)
    train_source_files = [os.path.join(source_folder,name) for name in train_names]
    train_dest = "data/train/"+waste_type
    move_files(train_source_files,train_dest)
    
    ## move source files to valid
    valid_names = get_names(waste_type,valid_ind)
    valid_source_files = [os.path.join(source_folder,name) for name in valid_names]
    valid_dest = "data/valid/"+waste_type
    move_files(valid_source_files,valid_dest)
    
    ## move source files to test
    test_names = get_names(waste_type,test_ind)
    test_source_files = [os.path.join(source_folder,name) for name in test_names]
    ## I use data/test here because the images can be mixed up
    move_files(test_source_files,"data/test")

In [0]:
## get a path to the folder with images
path = Path(os.getcwd())/"data"
path

In [0]:
tfms = get_transforms(do_flip=True,flip_vert=True)
data = ImageDataBunch.from_folder(path,test="test",ds_tfms=tfms,bs=16)

In [0]:
data

In [0]:
print(data.classes)

In [0]:
data.show_batch(rows=4,figsize=(10,8))

## 3. Model training

In [0]:
learn = cnn_learner(data,models.resnet34,metrics=error_rate)

In [0]:
print(learn)

In [0]:
from torchviz import make_dot, make_dot_from_trace
from torch.autograd import Variable

inputs = torch.randn(1,3,224,224)
resnet34 = models.resnet34()
y = resnet34(Variable(inputs))
# print(y)

dot = make_dot(y)
dot.format = 'pdf'
dot.render("struct")

### Finding a learning rate



In [0]:
def find_appropriate_lr(model:Learner, lr_diff:int = 15, loss_threshold:float = .05, adjust_value:float = 1, plot:bool = True) -> float:
    #Run the Learning Rate Finder
    model.lr_find()
    
    #Get loss values and their corresponding gradients, and get lr values
    losses = np.array(model.recorder.losses)
    assert(lr_diff < len(losses))
    loss_grad = np.gradient(losses)
    lrs = model.recorder.lrs
    
    #Search for index in gradients where loss is lowest before the loss spike
    #Initialize right and left idx using the lr_diff as a spacing unit
    #Set the local min lr as -1 to signify if threshold is too low
    r_idx = -1
    l_idx = r_idx - lr_diff
    while (l_idx >= -len(losses)) and (abs(loss_grad[r_idx] - loss_grad[l_idx]) > loss_threshold):
        local_min_lr = lrs[l_idx]
        r_idx -= 1
        l_idx -= 1

    lr_to_use = local_min_lr * adjust_value
    
    if plot:
        # plots the gradients of the losses in respect to the learning rate change
        plt.plot(loss_grad)
        plt.plot(len(losses)+l_idx, loss_grad[l_idx],markersize=10,marker='o',color='red')
        plt.ylabel("Loss")
        plt.xlabel("Index of LRs")
        plt.show()

        plt.plot(np.log10(lrs), losses)
        plt.ylabel("Loss")
        plt.xlabel("Log 10 Transform of Learning Rate")
        loss_coord = np.interp(np.log10(lr_to_use), np.log10(lrs), losses)
        plt.plot(np.log10(lr_to_use), loss_coord, markersize=10,marker='o',color='red')
        plt.show()
        
    return lr_to_use

In [0]:
sugesstion=find_appropriate_lr(learn)

learn.lr_find(start_lr=1e-6,end_lr=1e1)
learn.recorder.plot()


### Training

In [0]:
learn.fit_one_cycle(34,max_lr=sugesstion)

In [0]:
learn.recorder.plot_lr()

In [0]:
learn.recorder.plot_lr(show_moms=True)

In [0]:
learn.recorder.plot_losses()

In [0]:
learn.recorder.plot_metrics()

In [0]:
learn.save("trained_model", return_path=True)

In [0]:
!cp /content/data/models/trained_model.pth /content/

### VIsualizing most incorrect images

In [0]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

In [0]:
interp.plot_top_losses(9, figsize=(15,11))

The images here that the recycler performed poorly on were actually degraded. It looks the photos received too much exposure or something so this actually isn't a fault with the model!

In [0]:
doc(interp.plot_top_losses)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

This model often confused plastic for glass and confused metal. The list of most confused images is below.

In [0]:
interp.most_confused(min_val=2)

## 4. Make new predictions on test data


In [0]:
preds = learn.get_preds(ds_type=DatasetType.Test)

In [0]:
print(preds[0].shape)
preds[0]

In [0]:
data.classes

Now I'm going to convert the probabilities in the tensor above to a string with one of the class names.

In [0]:
## saves the index (0 to 5) of most likely (max) predicted class for each image
max_idxs = np.asarray(np.argmax(preds[0],axis=1))

In [0]:
yhat = []
for max_idx in max_idxs:
    yhat.append(data.classes[max_idx])

In [0]:
yhat

These are the predicted labels of all the images! Let's check if the first image is actually glass.

In [0]:
learn.data.test_ds[0][0]

It is!

Next, I'll get the actual labels from the test dataset.

In [0]:
y = []

## convert POSIX paths to string first
for label_path in data.test_ds.items:
    y.append(str(label_path))
    
## then extract waste type from file path
pattern = re.compile("([a-z]+)[0-9]+")
for i in range(len(y)):
    y[i] = pattern.search(y[i]).group(1)

A quick check.

In [0]:
## predicted values
print(yhat[0:5])
## actual values
print(y[0:5])

In [0]:
learn.data.test_ds[0][0]

In [0]:
cm = confusion_matrix(y,yhat)
print(cm)

Let's try and make this matrix a little prettier.

In [0]:
df_cm = pd.DataFrame(cm,waste_types,waste_types)

plt.figure(figsize=(10,8))
sns.heatmap(df_cm,annot=True,fmt="d",cmap="YlGnBu")

Again, the model seems to have confused metal for glass and plastic for glass. With more time, I'm sure further investigation could help reduce these mistakes.

In [0]:
correct = 0

for r in range(len(cm)):
    for c in range(len(cm)):
        if (r==c):
            correct += cm[r,c]

In [0]:
accuracy = correct/sum(sum(cm))
accuracy

In [0]:
## delete everything when you're done to save space
shutil.rmtree("data")
shutil.rmtree('dataset-resized')

In [0]:
!ls -la

In [0]:
!date